# Multipage Tables Scrape Demo

You're often going to encounter data and tables that is spread across hundreds if not thousands of pages. 

We're going to scrape as a demo a table that runs across several pages on this mock website.

```https://sandeepmj.github.io/scrape-example-page/heaviest-animals-page1.html```

To capture your target information into a single CSV file will require the use of many of the foundational skills we've covered, including:

- ```delays```
- ```conditional logic```
- ```while loops```
- ```BeautifulSoup```


And we'll explore a few new functional Python methods today.

## Scraping Strategies

- How do we approach this scrape?
- What pattern do we see?
- How do we capture a table on a single page?
- How do we capture a sequence of tables?
- How we navigate from page 1 to the subsequent pages?

# Let's code!

In [1]:
# import libraries

from bs4 import BeautifulSoup  ## web scraping
import requests ## request html for a page(s)
import csv ## read or write to csv
import pandas as pd ## pandas to work with data
import re ## regular express in one of our functions

## Single Table Scrape

In [2]:
##scrape url website
url = "https://sandeepmj.github.io/scrape-example-page/heaviest-animals-page1.html"
page = requests.get(url)
print(page.status_code)  ## should print 200. checks http response code status

200


In [3]:
## turn into soup
soup = BeautifulSoup(page.content, "html.parser")
print(type(soup))

<class 'bs4.BeautifulSoup'>


In [4]:
## MUST turn html into a string
html = soup.prettify()
print(type(html))

<class 'str'>


In [5]:
## use Pandas to read tables on page
df = pd.read_html(html)
df

[                 Animal  Weight(kg)    Type
 0            Blue whale      136000  Marine
 1         Bowhead whale      100000  Marine
 2             Fin whale       70000  Marine
 3  Southern right whale       45000  Marine
 4        Humpback whale       30000  Marine]

In [ ]:
## Do we want the first table?


In [6]:
## store it into a copy called animals_df
animals_df = df[0].copy() 
animals_df

,Animal,Weight(kg),Type
0,Blue whale,136000,Marine
1,Bowhead whale,100000,Marine
2,Fin whale,70000,Marine
3,Southern right whale,45000,Marine
4,Humpback whale,30000,Marine


## But we want to scrape multiple pages
2 ways to build a list of urls that we have to navigate to:

1. Placeholders
2. f-strings

### 1. Placeholders

In [7]:
## How is it different?
url = "https://sandeepmj.github.io/scrape-example-page/heaviest-animals-page{}.html"

## Placeholders

<img src="../support_files/placeholder1.png" style="width:500px;">

## Placeholders

<img src="../support_files/placeholder2.png" style="width:500px;">

## Placeholders

<img src="../support_files/placeholder3.png" style="width:500px;">

## Filling the Placeholder

### We use ```.format()``` to fill in values into the ```{}```placeholder

In [8]:
## here's our base url
## here's our base url
base_link = "http://www.example{}.html"
base_link

'http://www.example{}.html'

In [11]:
## Using a ```for loop```

all_urls_fl = []
for url_number in range(1,7):
    print(url_number)
    print(base_link.format(url_number))
    all_urls_fl.append(base_link.format(url_number))

1
http://www.example1.html
2
http://www.example2.html
3
http://www.example3.html
4
http://www.example4.html
5
http://www.example5.html
6
http://www.example6.html


In [12]:
all_urls_fl

['http://www.example1.html',
 'http://www.example2.html',
 'http://www.example3.html',
 'http://www.example4.html',
 'http://www.example5.html',
 'http://www.example6.html']

In [13]:
## using list comprehension
all_urls_lc = [base_link.format(url_number) for url_number in range(1,7)]
all_urls_lc

['http://www.example1.html',
 'http://www.example2.html',
 'http://www.example3.html',
 'http://www.example4.html',
 'http://www.example5.html',
 'http://www.example6.html']

### 2. Using f-strings

In [14]:
burl = "http://www.example"


In [15]:
## Using a ```for loop```
all_urls_fs = []
for url_number in range(1,7):
    temp_url = f"{burl}{url_number}.html"
    all_urls_fs.append(temp_url)

In [16]:
all_urls_fs

['http://www.example1.html',
 'http://www.example2.html',
 'http://www.example3.html',
 'http://www.example4.html',
 'http://www.example5.html',
 'http://www.example6.html']

## Back to our scrape

In [17]:
## let's remind ourselves of url variable's value

url

'https://sandeepmj.github.io/scrape-example-page/heaviest-animals-page{}.html'

## We know we need a placeholder value of upto ```4```
## Let's create a variable called  ```total_pages``` to match number of pages on site.

In [19]:
## total pages to scrape
total_pages = 5

In [24]:
## Let's write the for loop
## but instead of storing into a list, we just feed it directly to our placeholder
## we want to just scape each page
for url_number in range(1,total_pages):
    link = url.format(url_number)
    site = requests.get(link)
    print(site.status_code)

200
200
200
200


# We have a problem...

### We're hitting the server way too fast. We have to add a delay before we proceed.

In [25]:
## Let's import the required libaries to create a delay
from random import randrange ##  allows us to randomize numbers library
import time ## time tracker

In [26]:
## Let's run our code again but with appropriate delay

for url_number in range(1,total_pages):
    link = url.format(url_number)
    site = requests.get(link)
    print(site.status_code)
    snooze = randrange(3,6)
    print(f"snoozing for {snooze} seconds before next url")
    time.sleep(snooze)
    

200
snoozing for 3 seconds before next url
200
snoozing for 3 seconds before next url
200
snoozing for 5 seconds before next url
200
snoozing for 5 seconds before next url


## Working Around Errors

When you scrape hundreds of pages, there's chance that one of the URLs might be a dud.

We can set up a error control to see what kind of responses we get:

```<Response [200]>``` means website is accessible.

```<Response [404]>``` means broken link or no page on content.

In that case, your whole code might break and you'll have to figure out where it broke.

We can make that easier with conditional logic.

In [28]:
## CHECK FOR ERROR
for url_number in range(0,total_pages):
    link = url.format(url_number)
    site = requests.get(link)
    print(site.status_code)
    if site.status_code == 200:
        soup = BeautifulSoup(site.content, "html.parser")
        snooze = randrange(3,6)
        print(f"snoozing for {snooze} seconds before next url")
        time.sleep(snooze)
        
    else: 
        print(f"oh no, {link} returned: ", site.status_code)

404
oh no, https://sandeepmj.github.io/scrape-example-page/heaviest-animals-page0.html returned:  404
200
snoozing for 4 seconds before next url
200
snoozing for 3 seconds before next url
200
snoozing for 5 seconds before next url
200
snoozing for 3 seconds before next url


# All in One Step

Because we are using a  ```for loop``` that cycles through each link to do multiple steps on our target data, we need to have it done as one step.



In [29]:
## Combined url timed nav with table scrape
df_all = []
for url_number in range(0,total_pages):
    link = url.format(url_number)
    site = requests.get(link)
    print(site.status_code)
    if site.status_code == 200:
        soup = BeautifulSoup(site.content, "html.parser")
        html = soup.prettify()  ## turn soup into html 
        df = pd.read_html(html) ## turn html table into a df using pandas
        df_all.append(df[0]) ## append that first table to a list
        snooze = randrange(3,6)
        print(f"snoozing for {snooze} seconds before next url")
        time.sleep(snooze)
        
    else: 
        print(f"oh no, {link} returned: ", site.status_code)
print("Done scraping")

404
oh no, https://sandeepmj.github.io/scrape-example-page/heaviest-animals-page0.html returned:  404
200
snoozing for 5 seconds before next url
200
snoozing for 5 seconds before next url
200
snoozing for 4 seconds before next url
200
snoozing for 5 seconds before next url


In [33]:
df_all

[                 Animal  Weight(kg)    Type
 0            Blue whale      136000  Marine
 1         Bowhead whale      100000  Marine
 2             Fin whale       70000  Marine
 3  Southern right whale       45000  Marine
 4        Humpback whale       30000  Marine,
                  Animal  Weight(kg)    Type
 0            Gray whale       28500  Marine
 1  Northern right whale       23000  Marine
 2             Sei whale       20000  Marine
 3         Bryde's whale       16000  Marine
 4  Baird's beaked whale       11380  Marine,
                       Animal  Weight(kg)         Type
 0                Minke whale        7500       Marine
 1  Northern bottlenose whale        6500       Marine
 2     Gervais's beaked whale        5600       Marine
 3           African elephant        4800  Terrestrial
 4               Killer whale        3988       Marine,
                      Animal  Weight(kg)         Type
 0              Hippopotamus        3750  Terrestrial
 1            Asian

In [36]:
## FUNCTION to download individual dataframes in a list as a single csv
def combine_tables(list_name,filename):
  '''
  Takes dataframes in a list and combines into a single CSV.
  Tables must have identical column headers and order
  Arguments: name of list produced by tabula and the CSV name you want (in quotes as a string)
  '''
  dataframes = [pd.DataFrame(a_table) for a_table in list_name] ## list comprehension to turn each tabula table into a dataframe
  df = pd.concat(dataframes) ## join/concat all the dataframes into one dataframe
  df.reset_index(inplace = True, drop = True) ## reset index, drop what was there before
  df.to_csv(filename, encoding='utf-8', index=False) ## convert that single dataframe into a csv
#   files.download(filename) ## download it
  print(f"{filename} is in your current folder!")

In [37]:
## CALL THE FUNCTION
combine_tables(df_all, "heavy_animals.csv")

heavy_animals.csv is in your current folder!
